In [15]:
import os

os.environ["DATABRICKS_CONFIG_PROFILE"] = "C:/Users/feldm"

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.sql import ExecuteStatementRequestOnWaitTimeout
from databricks.sdk.service import ml

import time
from datetime import datetime, timezone
import uuid

import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import numpy as np
import optuna

In [16]:
def fetch_historical_data(workspace_client):

    warehouse = w.warehouses.list()[0]
    print(f"Using warehouse: {warehouse.name} ({warehouse.id}); state={warehouse.state}")
    
    SCHEMA_NAME = 'default'
    CATALOG_NAME = 'workspace'
    TABLE_NAME = 'aapl_market_price'
    
    response = w.statement_execution.execute_statement(
        statement=f"SELECT * FROM {CATALOG_NAME}.{SCHEMA_NAME}.{TABLE_NAME}",
        warehouse_id=warehouse.id,
        wait_timeout="30s",
        on_wait_timeout=ExecuteStatementRequestOnWaitTimeout.CANCEL  # Cancel if timeout
    )
    
    column_names = [col.name for col in response.manifest.schema.columns]
    df_original = pd.DataFrame(response.result.data_array, columns=column_names)

    return df_original

In [17]:
w = WorkspaceClient(profile="DEFAULT", config_file="C:/Users/feldm/.databrickscfg")

df_original = fetch_historical_data(w)

Using warehouse: Serverless Starter Warehouse (fe0c99db2c996651); state=State.RUNNING


In [18]:
for col in ['adj_open', 'adj_high', 'adj_low', 'adj_close', 'adj_volume']:
    df_original[col] = df_original[col].astype(float)

df_original.drop(['open', 'high', 'low', 'close', 'volume', 'split_factor', 'dividend'], axis=1, inplace=True)

In [19]:
# --- 1) Create an experiment ---
exp_name = f"/Users/feldmanngreg@gmail.com/my-experiment-{uuid.uuid4().hex[:8]}"  # Unique name
exp = w.experiments.create_experiment(name=exp_name)
exp_id = exp.experiment_id
print(f"Experiment: {exp_id}")

# --- 2) Create a parent run with optional tags ---
parent_run = w.experiments.create_run(
    experiment_id=exp_id,
    run_name="stock-direction-predictor-parent-run"
)
parent_run_id = parent_run.run.info.run_id
print(f"Created parent run: {parent_run_id}")

# --- 3) Log initial params in a single batch for parent run ---
now_ms = int(time.time() * 1000)
w.experiments.log_batch(
    run_id=parent_run_id,
    params=[
        ml.Param(key="framework", value="statsmodels"),
        ml.Param(key="model_type", value="Logit"),
        ml.Param(key="train_test_split", value="0.8"),
        ml.Param(key="optuna_trials", value="20")
    ],
    metrics=[]  # Metrics will be logged after training
)
print("Logged initial params to parent run.")

Experiment: 2628800825290818
Created parent run: 8a225f8022de4136b679bf7787f2ff83
Logged initial params to parent run.


In [20]:
# Define Optuna objective for hyperparameter search (tuning number of lags)
def objective(trial):
    lags = trial.suggest_int('lags', 1, 10)  # Tune number of lags

    # Create a new run for this trial
    trial_run = w.experiments.create_run(
        experiment_id=exp_id,
        run_name=f"trial_{trial.number}"
    )
    trial_run_id = trial_run.run.info.run_id
    print(f"Created trial run: {trial_run_id}")

    # Log trial params
    w.experiments.log_batch(
        run_id=trial_run_id,
        params=[
            ml.Param(key="lags", value=str(lags)),
            ml.Param(key="framework", value="statsmodels"),
            ml.Param(key="model_type", value="Logit"),
            ml.Param(key="train_test_split", value="0.8"),
            ml.Param(key="optuna_trials", value="20")
        ]
    )

    # Re-generate features with current lags
    df = df_original.copy()
    for i in range(1, lags + 1):
        df[f'lag_{i}'] = df['adj_close'].shift(i)

    # Target: Next day's adj_close for comparison
    df['target'] = df['adj_close'].shift(-1)

    # Binary target: 1 if next adj_close > current adj_close (up), else 0 (down)
    df['up_down'] = (df['target'] > df['adj_close']).astype(int)

    # Drop rows with NaN (from shifts)
    df.dropna(inplace=True)

    # Exogenous features: adj columns + lags (added adj_close as per user update)
    features = ['adj_open', 'adj_high', 'adj_low', 'adj_volume', 'adj_close'] + [f'lag_{i}' for i in range(1, lags + 1)]
    
    # Ensure all features are numeric (redundant if df_raw converted, but safe)
    df[features] = df[features].apply(pd.to_numeric, errors='coerce')
    df.dropna(inplace=True)  # Drop any coerced NaNs

    exog = df[features]
    endog = df['up_down']  # Binary endogenous variable

    # Split data: Time-series split (80% train, 20% test)
    train_size = int(len(df) * 0.8)
    exog_train, exog_test = exog.iloc[:train_size], exog.iloc[train_size:]
    endog_train, endog_test = endog.iloc[:train_size], endog.iloc[train_size:]

    # Train statsmodels Logit
    exog_train_const = sm.add_constant(exog_train)
    model = sm.Logit(endog_train, exog_train_const)
    model_fit = model.fit(disp=False, method='newton', maxiter=1000)  # Fixed method for consistency

    # Evaluate on test set
    exog_test_const = sm.add_constant(exog_test)
    prob_pred = model_fit.predict(exog_test_const)
    pred_class = (prob_pred > 0.5).astype(int)
    accuracy = (pred_class == endog_test).mean()

    # Log trial metric
    now_ms = int(time.time() * 1000)
    w.experiments.log_batch(
        run_id=trial_run_id,
        metrics=[
            ml.Metric(key="accuracy", value=accuracy, timestamp=now_ms, step=0)
        ]
    )
    print(f"Logged trial {trial.number} with accuracy: {accuracy}")

    return accuracy  # Maximize accuracy on validation (here test, but for real CV, average folds)

In [21]:
# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # Adjust n_trials as needed

best_params = study.best_params
best_accuracy = study.best_value
print(f'Best params: {best_params}')
print(f'Best accuracy: {best_accuracy}')

[I 2025-08-17 15:24:12,182] A new study created in memory with name: no-name-3bd241f1-047e-419c-85b5-c015f7f51d4e


Created trial run: 109757935d7d42c18890e20581a2f7c2


[I 2025-08-17 15:24:12,512] Trial 0 finished with value: 0.52690084481992 and parameters: {'lags': 6}. Best is trial 0 with value: 0.52690084481992.


Logged trial 0 with accuracy: 0.52690084481992
Created trial run: 2c5f3cae2798433b9b29b69787fa2917


[I 2025-08-17 15:24:12,869] Trial 1 finished with value: 0.5253333333333333 and parameters: {'lags': 2}. Best is trial 0 with value: 0.52690084481992.


...

Logged trial 17 with accuracy: 0.5264562027567808
Created trial run: 7f0f38a4133d4380a90dae332f3fd6cc


[I 2025-08-17 15:24:18,299] Trial 18 finished with value: 0.5264562027567808 and parameters: {'lags': 7}. Best is trial 0 with value: 0.52690084481992.


Logged trial 18 with accuracy: 0.5264562027567808
Created trial run: 72d8c1c01e1b4e61811821a102706c7f


[I 2025-08-17 15:24:18,615] Trial 19 finished with value: 0.5246776345042241 and parameters: {'lags': 3}. Best is trial 0 with value: 0.52690084481992.


Logged trial 19 with accuracy: 0.5246776345042241
Best params: {'lags': 6}
Best accuracy: 0.52690084481992


In [23]:
# Re-train final model with best params (best lags)
lags = best_params['lags']
df = df_original.copy()  # Assuming df_original is df_raw
for i in range(1, lags + 1):
    df[f'lag_{i}'] = df['adj_close'].shift(i)
df['target'] = df['adj_close'].shift(-1)
df['up_down'] = (df['target'] > df['adj_close']).astype(int)
df.dropna(inplace=True)
features = ['adj_open', 'adj_high', 'adj_low', 'adj_volume', 'adj_close'] + [f'lag_{i}' for i in range(1, lags + 1)]
exog = df[features]
endog = df['up_down']
train_size = int(len(df) * 0.8)
exog_train, exog_test = exog.iloc[:train_size], exog.iloc[train_size:]
endog_train, endog_test = endog.iloc[:train_size], endog.iloc[train_size:]
exog_train_const = sm.add_constant(exog_train)
model = sm.Logit(endog_train, exog_train_const)
model_fit = model.fit(disp=False)
exog_test_const = sm.add_constant(exog_test)
prob_pred = model_fit.predict(exog_test_const)
pred_class = (prob_pred > 0.5).astype(int)
accuracy = (pred_class == endog_test).mean()
print(f'Accuracy on test set: {accuracy}')
# Log metrics and best params after training
log_params = [ml.Param(key=k, value=str(v)) for k, v in best_params.items()]
now_ms = int(time.time() * 1000)
w.experiments.log_batch(
    run_id=parent_run_id,
    params=log_params,
    metrics=[
        ml.Metric(key="optuna_best_accuracy", value=best_accuracy, timestamp=now_ms, step=0),
        ml.Metric(key="test_accuracy", value=accuracy, timestamp=now_ms, step=0)
    ]
)
print("Logged metrics and best params.")
# --- 4) Create a logged model tied to that run ---
logged_model_params = [ml.LoggedModelParameter(key=k, value=str(v)) for k, v in best_params.items()]
resp = w.experiments.create_logged_model(
    experiment_id=exp_id,
    source_run_id=parent_run_id,
    model_type="Classifier",
    name="stock-direction-model",
    params=logged_model_params + [
        ml.LoggedModelParameter(key="framework", value="statsmodels"),
    ],
    tags=[
        ml.LoggedModelTag(key="owner", value="feldmanngreg@gmail.com"),
        ml.LoggedModelTag(key="env", value="dev"),
    ]
)
print("CreateLoggedModelResponse:", resp)
# --- 5) Read the run back (includes latest metrics/params/tags) ---
got = w.experiments.get_run(run_id=parent_run_id)
print("Params:", {p.key: p.value for p in got.run.data.params})
print("Metrics:", {m.key: m.value for m in got.run.data.metrics})
print("Tags:", {t.key: t.value for t in got.run.data.tags})

Accuracy on test set: 0.52690084481992
Logged metrics and best params.
CreateLoggedModelResponse: CreateLoggedModelResponse(model=LoggedModel(data=LoggedModelData(metrics=[], params=[LoggedModelParameter(key='framework', value='statsmodels'), LoggedModelParameter(key='lags', value='6')]), info=LoggedModelInfo(artifact_uri='dbfs:/databricks/mlflow-tracking/2628800825290818/logged_models/m-3e3dd428b87c48ecab44de803e285734/artifacts', creation_timestamp_ms=1755410063632, creator_id=76055929257863, experiment_id='2628800825290818', last_updated_timestamp_ms=1755410063637, model_id='m-3e3dd428b87c48ecab44de803e285734', model_type='Classifier', name='stock-direction-model', source_run_id='8a225f8022de4136b679bf7787f2ff83', status=<LoggedModelStatus.LOGGED_MODEL_PENDING: 'LOGGED_MODEL_PENDING'>, status_message=None, tags=[LoggedModelTag(key='env', value='dev'), LoggedModelTag(key='mlflow.loggedModel.artifactLocation', value='dbfs:/databricks/mlflow-tracking/2628800825290818/logged_models/m-3e